# Notebook 04: Multi-Connector Optimization with Explicit Summarization Prompts

## Objective

In this notebook, we'll focus on demonstrating the optimization capabilities of our multi-connector setup. We'll use a simple, explicit summarization prompt to illustrate how the system can optimize the selection of connectors for specific tasks by offloading the summarization skill from ChatGPT to smaller Oobabooga models.

## Prerequisites

- Before proceeding, make sure you have run `00-AI-settings.ipynb` to set up your OpenAI key, since we'll use ChatGPT as our primary connector, and have at least one Oobabooga model running and configured in the `Multiconnector` section of your `settings.json` file according to your VRam capabilities.
- Basic knowledge of the multi-connector pipeline from `03-multiConnector-intro-with-arithmetic-mocks.ipynb`.

## 1. Setup

### 1.1 Import Required Libraries

We'll start with importing the appropriate Nugget packages to load the configuration settings, and to run the Multiconnector itself. 

In [6]:
//Import package for loading hierarchichal settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Microsoft.Extensions.Configuration.Binder"

// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
// Import Oobabooga connector package
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga"
// Import Multiconnector package
#r "nuget: MyIA.SemanticKernel.Connectors.AI.MultiConnector"

Installed Packages Microsoft.Extensions.Configuration, 7.0.0 Microsoft.Extensions.Configuration.Binder, 7.0.4 Microsoft.Extensions.Configuration.Json, 7.0.0 Microsoft.SemanticKernel, 0.24.230918.1-preview MyIA.SemanticKernel.Connectors.AI.MultiConnector, 0.33.8 MyIA.SemanticKernel.Connectors.AI.Oobabooga, 0.33.8

### 1.2 Load Settings

We load OpenAi and Multiconnector configuration from the settings file

In [7]:
// Load configuration using builder package
using System.IO;
using Microsoft.Extensions.Configuration;
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

var openAIConfiguration = configuration.GetSection("OpenAI").Get<OpenAIConfiguration>();
var multiOobaboogaConnectorConfiguration = configuration.GetSection("MultiConnector").Get<MultiOobaboogaConnectorConfiguration>();

### 1.3 Set up MultiTextCompletion settings

There are many parameters controlling how the multiconnector will work and perform optimization. We need to create an instance of the corresponding class.

Also, because we'll be measuring costs to perform our optimization, we need to create an creditor object dedicated to that.

For now, we'll stick to the default parameters.

In [8]:
using MyIA.SemanticKernel.Connectors.AI.MultiConnector;
using System.Text.Json;
using System.Text.Json.Serialization;
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.PromptSettings;

var creditor = new CallRequestCostCreditor();

// The most common settings for a MultiTextCompletion are illustrated below, most of them have default values and are optional
var settings = new MultiTextCompletionSettings()
{
    Creditor = creditor
};

string jsonString = JsonSerializer.Serialize(settings, new JsonSerializerOptions() { WriteIndented = true });
display(jsonString);

{
  "FreezePromptTypes": false,
  "PromptTruncationLength": 20,
  "AdjustPromptStarts": false,
  "EnablePromptSampling": true,
  "MaxInstanceNb": 10,
  "AnalysisSettings": {
    "EnableAnalysis": false,
    "AnalysisFilePath": ".\\MultiTextCompletion-analysis.json",
    "AnalysisDelay": "00:00:01",
    "AnalysisAwaitsManualTrigger": false,
    "EnableConnectorTests": true,
    "TestPrimaryCompletion": true,
    "TestsPeriod": "00:00:10",
    "MaxDegreeOfParallelismTests": 1,
    "MaxDegreeOfParallelismConnectorsByTest": 3,
    "EnableTestEvaluations": true,
    "EvaluationPeriod": "00:00:10",
    "MaxDegreeOfParallelismEvaluations": 5,
    "UseSelfVetting": false,
    "EnableSuggestion": true,
    "SuggestionPeriod": "00:01:00",
    "UpdateSuggestedSettings": true,
    "SaveSuggestedSettings": false,
    "DeleteAnalysisFile": true,
    "MultiCompletionSettingsFilePath": ".\\MultiTextCompletionSettings.json",
    "NbPromptTests": 3,
    "VettingPromptTransform": {
      "Template": "Val

## 2. Initialization

With all the settings created, we can now create the semantic kernel that we'll use to run our tests.

### 2.1 Create primary and secondary completions

We'll use the OpenAi configuration to instantiate a text or chat based connector typically using ChatGPT.
Then we can use a helper method from the multicompletion configuration to instantiate all Oobabooga secondary connectors.

In [15]:
using System.Threading;
using Microsoft.SemanticKernel.AI.TextCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextCompletion;

// Creating a cancellation token source to be able to cancel the request
CancellationTokenSource cleanupToken = new();

//Creating the primary connector. We use the OpenAI connector here, either text or chat completion depending on the configuration
 ITextCompletion openAiConnector;

 string testOrChatModelId;
 if (openAIConfiguration.ChatModelId != null)
 {
     testOrChatModelId = openAIConfiguration.ChatModelId;
     openAiConnector = new OpenAIChatCompletion(testOrChatModelId, openAIConfiguration.ApiKey);
 }
 else
 {
     testOrChatModelId = openAIConfiguration.ModelId;
     openAiConnector = new OpenAITextCompletion(testOrChatModelId, openAIConfiguration.ApiKey);
 }

 // Creating the corresponding named completion
 var openAiNamedCompletion = new NamedTextCompletion(testOrChatModelId, openAiConnector)
{
    MaxTokens = openAIConfiguration.MaxTokens,
    CostPer1000Token = openAIConfiguration.CostPer1000Token,
    TokenCountFunc = MultiOobaboogaConnectorConfiguration.TokenCountFunctionMap[openAIConfiguration.TokenCountFunction],
    //We did not observe any limit on Open AI concurrent calls
    MaxDegreeOfParallelism = 5,
};

 // Creating the secondary connectors. We use a dedicated helper, but you can create them manually if you want.
 var oobaboogaCompletions = multiOobaboogaConnectorConfiguration.CreateNamedCompletions();

 display($"Number of secondary completions created: {oobaboogaCompletions.Count}");

Number of secondary completions created: 2

### 2.2 Create Kernel

Now that we have our primary and secondary completions, we can create a semantic kernel and instantiate our multiconnector from settings and completions.

We use the dedicated helper for that.

In [16]:
var builder = Microsoft.SemanticKernel.Kernel.Builder;

builder.WithMultiConnectorCompletionService(
    serviceId: null,
    settings: settings,
    mainTextCompletion: openAiNamedCompletion,
    setAsDefault: true,
    analysisTaskCancellationToken: cleanupToken.Token,
    otherCompletions: oobaboogaCompletions.ToArray());

var kernel = builder.Build();

### 2.3 Create simple inline semantic function

For this first example, we concentrate on offloading a single simple parameter-less semantic function. 

In the next notebook, we'll move on with considering skills, inputs of various complexities, static and finally dynamic plans.

In [26]:
using Microsoft.SemanticKernel;

var text = @"A long time ago, people wanted to tell others their stories. First, they wrote letters with their hands. They would send these letters to friends far away. Sometimes, people waited a lot of days to get a letter.

After that, a big machine called the printing press was made. It could make many copies of a story quickly. More people could read the same thing without waiting.

Next, there was a telephone. With it, people could talk and listen to friends who were far. They didn’t have to wait for letters anymore.

Then, there was a thing called television. People could watch stories on it, like a play. They didn’t need to go outside.

Lastly, came mobile phones and computers. People could send messages fast. With the internet, they could also use something called social media to share stories with many people at once.";

var prompt = $"Summarize the following text in one sentence:\n{text}\n\nSummary:";

var simpleSemanticFunction = kernel.CreateSemanticFunction(prompt, maxTokens: 100);

## 3. Running and optimizing settings

Now that everything is in order we'll follow the following workflow:

- The plan is run once. The primary connector defined (Chat GPT) is used to generate our completion.
    - Performance in cost and in duration is recorded.
    - Samples are collected automatically during the run
    - Result of the plan is shown.
- An analysis task is run from samples collected during the run.
    - Each connector is tested on the samples.
    - The primary connector (ChatGPT) evaluates the test runs, vetting each connector's capability to handle each corresponding prompt type.
    - New settings are computed from the evaluation. Vetted connectors are promoted to handle the corresponding prompt types.
    - MultiCompletion settings are updated according to the analysis results.
- The original plan is reloaded and run again. This time, the secondary connectors may be used to generate some or all of the completions according to the updated settings.
    - Performance in cost and in duration is recorded.
    - Result of the plan is shown

### 3.1 Run function with Primary Connector

For this first example, we want our multiconnector to do the job automatically for us, so we'll configure our settings accordingly before we run our function.

In [ ]:


settings.EnablePromptSampling = true;
settings.AnalysisSettings.EnableAnalysis = true;